In [83]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time


In [84]:
def bypass_cookie_overlay(driver):
    time.sleep(3)
    driver.find_element_by_class_name('css-flk0bs').click()
    teamsLi = driver.find_element_by_id('header_teams')
    button = teamsLi.find_element_by_tag_name('a')
    button.click()

In [85]:
def get_all_games_from_season(season):
    games = []
    driver = webdriver.Firefox()
    driver.get('https://basketball-reference.com')
    bypass_cookie_overlay(driver)
    seasons = driver.find_element_by_link_text('Seasons')
    seasons.click()
    season = driver.find_element_by_link_text(season)
    season.click()
    schedule = driver.find_element_by_link_text('Schedule and Results')
    schedule.click()
    # There are 10 different months so we have to go through them all 
    filter = driver.find_element_by_class_name('filter')
    months = filter.text.split(' ')
    for month in months:
        # Get monthly data
        link = driver.find_element_by_link_text(month)
        link.click()
        table = driver.find_element_by_id('schedule')
        tbody = table.find_element_by_tag_name('tbody')
        datas = tbody.find_elements_by_tag_name('tr')
        for data in datas:
            game = {}
            tds = data.find_elements_by_tag_name('td')
            try:
                game['home'] = tds[1].text
                game['visitor'] = tds[3].text
                home_score = tds[2].text
                visitor_score = tds[4].text
                game['score']= home_score + '-' + visitor_score
                if(game['score']=='-' or game['score']==''):
                    break
                else:
                    games.append(game)
            except IndexError:
                continue
    if(games[-1]['score']=='-'):
        games.pop()
    return games 

In [87]:
def get_latest_nba_games():
    driver = webdriver.Firefox()
    driver.get("https://basketball-reference.com")
    scores = driver.find_element_by_id('scores')
    games = scores.find_elements_by_class_name('teams')
    rtn = []
    for game in games:
        gameData = {}
        temp = game.find_element_by_class_name('loser')
        loser = temp.find_element_by_tag_name('a').text
        loser_score = temp.find_element_by_class_name('right').text
        temp = game.find_element_by_class_name('winner')
        winner = temp.find_element_by_tag_name('a').text
        winner_score = temp.find_element_by_class_name('right').text
        gameData['loser'] = loser
        gameData['winner'] = winner
        gameData['score']= loser_score+'-'+winner_score
        print(gameData)
        rtn.append(gameData)
    driver.quit()
    return rtn


        

In [140]:
def get_team_stats(team_name,get_labels=False,home=''):
    driver = webdriver.Firefox()
    driver.get("https://basketball-reference.com")
    bypass_cookie_overlay(driver)
    try:
        driver.find_element_by_link_text(team_name).click()
    except:
        driver.find_element_by_partial_link_text(team_name).click()
    driver.find_element_by_link_text('2020-21').click()
    team_opponent = driver.find_element_by_id('team_and_opponent')
    tbody = team_opponent.find_element_by_tag_name('tbody')
    stats_row = tbody.find_elements_by_tag_name('tr')[1]
    stats = stats_row.find_elements_by_tag_name('td')[1:]
    labels = []
    values =[]
    for stat in stats:
        name_of_stat = stat.get_attribute('data-stat')
        if(home != ''):
            name_of_stat += '_'+home
        labels.append(name_of_stat)
        values.append(stat.text)
    time.sleep(3)
    driver.quit()
    if(get_labels):
        return labels, values
    else:
        return values
    


In [146]:
def get_latest_nba_games_with_stats():
    driver = webdriver.Firefox()
    driver.get("https://basketball-reference.com")
    scores = driver.find_element_by_id('scores')
    games = scores.find_elements_by_class_name('teams')
    
    rtn = []
    for game in games:
#         gameData = []
        temps = game.find_elements_by_tag_name('tr')
        home = temps[1].find_element_by_tag_name('a').text
        home_score = temps[1].find_element_by_class_name('right').text
        away = temps[0].find_element_by_tag_name('a').text
        away_score = temps[0].find_element_by_class_name('right').text
        labels_home, home_stats = get_team_stats(home, True,'HOME')
        labels_away, away_stats = get_team_stats(away,True,'AWAY')
        home_win =( [1] if home_score > away_score else [0])
        gameData = home_stats + away_stats + home_win
        labels = labels_home + labels_away + ['Home_win']
        if(len(rtn)==0):
            rtn.append(labels)
#         gameData['score']= loser_score+'-'+winner_score
        rtn.append(gameData)
    driver.quit()
    return rtn
    

In [168]:
data = get_latest_nba_games_with_stats()

[['mp_per_g_HOME', 'fg_per_g_HOME', 'fga_per_g_HOME', 'fg_pct_HOME', 'fg3_per_g_HOME', 'fg3a_per_g_HOME', 'fg3_pct_HOME', 'fg2_per_g_HOME', 'fg2a_per_g_HOME', 'fg2_pct_HOME', 'ft_per_g_HOME', 'fta_per_g_HOME', 'ft_pct_HOME', 'orb_per_g_HOME', 'drb_per_g_HOME', 'trb_per_g_HOME', 'ast_per_g_HOME', 'stl_per_g_HOME', 'blk_per_g_HOME', 'tov_per_g_HOME', 'pf_per_g_HOME', 'pts_per_g_HOME', 'mp_per_g_AWAY', 'fg_per_g_AWAY', 'fga_per_g_AWAY', 'fg_pct_AWAY', 'fg3_per_g_AWAY', 'fg3a_per_g_AWAY', 'fg3_pct_AWAY', 'fg2_per_g_AWAY', 'fg2a_per_g_AWAY', 'fg2_pct_AWAY', 'ft_per_g_AWAY', 'fta_per_g_AWAY', 'ft_pct_AWAY', 'orb_per_g_AWAY', 'drb_per_g_AWAY', 'trb_per_g_AWAY', 'ast_per_g_AWAY', 'stl_per_g_AWAY', 'blk_per_g_AWAY', 'tov_per_g_AWAY', 'pf_per_g_AWAY', 'pts_per_g_AWAY', 'Home_win'], ['242.3', '38.9', '85.7', '.454', '10.0', '29.3', '.340', '29.0', '56.4', '.513', '16.3', '21.9', '.745', '10.4', '32.0', '42.5', '24.2', '7.9', '4.7', '15.6', '18.4', '104.1', '243.1', '43.3', '88.4', '.490', '13.0',

[['mp_per_g_HOME', 'fg_per_g_HOME', 'fga_per_g_HOME', 'fg_pct_HOME', 'fg3_per_g_HOME', 'fg3a_per_g_HOME', 'fg3_pct_HOME', 'fg2_per_g_HOME', 'fg2a_per_g_HOME', 'fg2_pct_HOME', 'ft_per_g_HOME', 'fta_per_g_HOME', 'ft_pct_HOME', 'orb_per_g_HOME', 'drb_per_g_HOME', 'trb_per_g_HOME', 'ast_per_g_HOME', 'stl_per_g_HOME', 'blk_per_g_HOME', 'tov_per_g_HOME', 'pf_per_g_HOME', 'pts_per_g_HOME', 'mp_per_g_AWAY', 'fg_per_g_AWAY', 'fga_per_g_AWAY', 'fg_pct_AWAY', 'fg3_per_g_AWAY', 'fg3a_per_g_AWAY', 'fg3_pct_AWAY', 'fg2_per_g_AWAY', 'fg2a_per_g_AWAY', 'fg2_pct_AWAY', 'ft_per_g_AWAY', 'fta_per_g_AWAY', 'ft_pct_AWAY', 'orb_per_g_AWAY', 'drb_per_g_AWAY', 'trb_per_g_AWAY', 'ast_per_g_AWAY', 'stl_per_g_AWAY', 'blk_per_g_AWAY', 'tov_per_g_AWAY', 'pf_per_g_AWAY', 'pts_per_g_AWAY', 'Home_win'], ['242.3', '38.9', '85.7', '.454', '10.0', '29.3', '.340', '29.0', '56.4', '.513', '16.3', '21.9', '.745', '10.4', '32.0', '42.5', '24.2', '7.9', '4.7', '15.6', '18.4', '104.1', '243.1', '43.3', '88.4', '.490', '13.0',

In [112]:
import pandas as pd

In [169]:
df = pd.DataFrame(data[1:],columns=data[0])

In [170]:
df

,mp_per_g_HOME,fg_per_g_HOME,fga_per_g_HOME,fg_pct_HOME,fg3_per_g_HOME,fg3a_per_g_HOME,fg3_pct_HOME,fg2_per_g_HOME,fg2a_per_g_HOME,fg2_pct_HOME,...,orb_per_g_AWAY,drb_per_g_AWAY,trb_per_g_AWAY,ast_per_g_AWAY,stl_per_g_AWAY,blk_per_g_AWAY,tov_per_g_AWAY,pf_per_g_AWAY,pts_per_g_AWAY,Home_win
0,242.3,38.9,85.7,.454,10.0,29.3,.340,29.0,56.4,.513,...,8.9,34.4,43.3,27.0,7.2,4.4,12.8,19.2,114.8,0
1,242.3,38.7,85.9,.450,11.7,33.1,.353,27.0,52.8,.511,...,10.4,33.1,43.6,26.9,7.9,4.9,15.0,18.1,109.7,1
2,240.8,44.5,91.7,.485,14.5,37.1,.392,30.0,54.6,.549,...,9.1,35.3,44.4,26.6,6.7,5.1,13.7,19.0,118.7,1
3,241.5,38.7,83.6,.463,12.7,36.2,.351,26.0,47.4,.549,...,9.0,34.4,43.4,22.5,6.2,4.3,12.1,19.3,112.1,0
4,242.3,42.6,88.7,.480,10.7,30.5,.350,31.9,58.2,.549,...,7.9,34.9,42.9,27.4,8.3,4.9,15.1,21.3,112.9,1
5,241.1,38.8,88.0,.441,12.2,35.6,.343,26.6,52.4,.508,...,9.4,31.8,41.2,25.4,7.2,4.8,13.4,19.5,114.1,1
6,240.0,41.8,86.8,.482,14.4,34.5,.417,27.5,52.3,.526,...,9.3,32.2,41.5,24.4,8.5,5.5,13.3,21.3,111.5,1


In [171]:
df.to_csv(r'./nba_data_04_05_21.csv')

In [88]:
games = get_latest_nba_games()

{'loser': 'Detroit Pistons', 'winner': 'Orlando Magic', 'score': '112-119'}
{'loser': 'Indiana Pacers', 'winner': 'Washington Wizards', 'score': '141-154'}
{'loser': 'New Orleans Pelicans', 'winner': 'Golden State Warriors', 'score': '108-123'}
{'loser': 'Portland Trail Blazers', 'winner': 'Atlanta Hawks', 'score': '114-123'}
{'loser': 'Chicago Bulls', 'winner': 'Philadelphia 76ers', 'score': '94-106'}
{'loser': 'Memphis Grizzlies', 'winner': 'New York Knicks', 'score': '104-118'}
{'loser': 'Denver Nuggets', 'winner': 'Los Angeles Lakers', 'score': '89-93'}
{'loser': 'San Antonio Spurs', 'winner': 'Utah Jazz', 'score': '99-110'}


In [5]:
games

[{'loser': 'Toronto Raptors', 'winner': 'New York Knicks', 'score': '103-120'},
 {'loser': 'Philadelphia 76ers',
  'winner': 'Milwaukee Bucks',
  'score': '94-132'},
 {'loser': 'Detroit Pistons', 'winner': 'Indiana Pacers', 'score': '109-115'},
 {'loser': 'Chicago Bulls', 'winner': 'Miami Heat', 'score': '101-106'},
 {'loser': 'New Orleans Pelicans',
  'winner': 'San Antonio Spurs',
  'score': '108-110'},
 {'loser': 'Los Angeles Lakers',
  'winner': 'Dallas Mavericks',
  'score': '93-108'},
 {'loser': 'Utah Jazz', 'winner': 'Minnesota Timberwolves', 'score': '96-101'},
 {'loser': 'Houston Rockets', 'winner': 'Denver Nuggets', 'score': '116-129'}]